In [1]:
import tensorflow as tf
import time
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def create_simple_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(256, 256, 3)))
    model.add(MaxPooling2D(pool_size=(5, 5)))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

def create_mobilenet_model():
    mobile_net = tf.keras.applications.mobilenet_v2.MobileNetV2(
        include_top=False,
        weights=None,
        input_shape=[256, 256, 3],
        pooling='max')

    kernel_initializer = tf.keras.initializers.glorot_uniform(seed=1337)
    model = tf.keras.Sequential()

    model.add(mobile_net)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer = kernel_initializer))

    return model

In [3]:
datasets = {
    'normal': './dataset/train',
    'modified': './dataset/train_modified',
    'rescaled': './dataset/train_rescaled'   
}

In [5]:
# Change this const to train another type of dataset ('modified' and 'rescaled')
TRAINING_TYPE = 'normal'

dataset_generator = ImageDataGenerator(
  data_format='channels_last',
  rescale = 1. / 255,
  validation_split = 0.4
)

train_batches = dataset_generator.flow_from_directory(
    batch_size=32,
    directory=datasets[TRAINING_TYPE],
    target_size=(256, 256),
    class_mode='binary',
    subset='training'
)

validation_batches = dataset_generator.flow_from_directory(
    batch_size=32,
    directory=datasets[TRAINING_TYPE],
    target_size=(256, 256),
    class_mode='binary',
    subset='validation'
)

Found 39253 images belonging to 2 classes.
Found 26168 images belonging to 2 classes.


In [6]:
# Balance weights for loss depending on rate between the two categories
unique, counts = np.unique(train_batches.classes, return_counts=True)
class_weigths = dict(zip(unique, np.true_divide(counts.sum(), 2*counts)))

In [7]:
model_simple = create_simple_model()

In [8]:
model_simple.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [9]:
model_simple.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 51, 51, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 51, 51, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                591904    
__________

In [10]:
start = time.time()
history = model_simple.fit_generator(train_batches,
                    epochs=30,
                    verbose=1,
                    steps_per_epoch=len(train_batches),
                    validation_data=validation_batches,
                    initial_epoch=0,
                    class_weight=class_weigths,
                    validation_steps=len(validation_batches)
                   ) 
end = time.time()
print('Time to train: %.2fmin' % ((end - start) / 60))

Epoch 1/30
 305/1227 [======>.......................] - ETA: 9:59 - loss: 0.6759 - acc: 0.6307

KeyboardInterrupt: 

In [ ]:
model_dir = './h5/'
model_simple.save(model_dir + 'model_' + TRAINING_TYPE + '.h5')

history_dir = './history/'


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

epochs = history.epoch
acc = history.history['val_acc']

fig, ax = plt.subplots()
ax.plot(epochs, acc)
ax.grid(True, linestyle='-.')
ax.set_xlabel('epochs')
ax.set_ylabel('accuracy')


plt.show()

In [ ]:
test_generator = ImageDataGenerator(
    data_format='channels_last',
    rescale=1./255
)

test_batches = test_generator.flow_from_directory(
    batch_size=1,
    directory='./dataset/test_modified',
    target_size=[256, 256],
    class_mode='binary'
)

In [ ]:
score = model.evaluate_generator(test_batches, verbose=1)

print(model.metrics_names)
print('test dataset: ' + str(score))